<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Splitting-Large-Data-Sets-for-Parallel-Processing-of-Queries" data-toc-modified-id="Splitting-Large-Data-Sets-for-Parallel-Processing-of-Queries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Splitting Large Data Sets for Parallel Processing of Queries</a></span><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Prerequisites" data-toc-modified-id="Prerequisites-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Prerequisites</a></span></li><li><span><a href="#Setup" data-toc-modified-id="Setup-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Setup</a></span><ul class="toc-item"><li><span><a href="#Ensure-database-is-running" data-toc-modified-id="Ensure-database-is-running-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Ensure database is running</a></span></li><li><span><a href="#Download-and-install-additional-components." data-toc-modified-id="Download-and-install-additional-components.-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Download and install additional components.</a></span></li><li><span><a href="#Initialize-Client" data-toc-modified-id="Initialize-Client-1.3.3"><span class="toc-item-num">1.3.3&nbsp;&nbsp;</span>Initialize Client</a></span><ul class="toc-item"><li><span><a href="#Initialize-event-loops-for-async-processing-mode" data-toc-modified-id="Initialize-event-loops-for-async-processing-mode-1.3.3.1"><span class="toc-item-num">1.3.3.1&nbsp;&nbsp;</span>Initialize event loops for async processing mode</a></span></li><li><span><a href="#Initialize-client-with-event-loops" data-toc-modified-id="Initialize-client-with-event-loops-1.3.3.2"><span class="toc-item-num">1.3.3.2&nbsp;&nbsp;</span>Initialize client with event loops</a></span></li></ul></li><li><span><a href="#Includes-and-Constants" data-toc-modified-id="Includes-and-Constants-1.3.4"><span class="toc-item-num">1.3.4&nbsp;&nbsp;</span>Includes and Constants</a></span></li><li><span><a href="#Populate-Test-Data." data-toc-modified-id="Populate-Test-Data.-1.3.5"><span class="toc-item-num">1.3.5&nbsp;&nbsp;</span>Populate Test Data.</a></span></li></ul></li></ul></li><li><span><a href="#Overview" data-toc-modified-id="Overview-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Overview</a></span></li><li><span><a href="#Dividing-Partitions-into-Requested-Splits" data-toc-modified-id="Dividing-Partitions-into-Requested-Splits-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Dividing Partitions into Requested Splits</a></span></li><li><span><a href="#Parallel-Query-Framework" data-toc-modified-id="Parallel-Query-Framework-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Parallel Query Framework</a></span><ul class="toc-item"><li><span><a href="#Global-parameters" data-toc-modified-id="Global-parameters-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Global parameters</a></span></li></ul></li><li><span><a href="#Running-Parallel-Queries-with-Interesting-Parameter-Variations" data-toc-modified-id="Running-Parallel-Queries-with-Interesting-Parameter-Variations-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Running Parallel Queries with Interesting Parameter Variations</a></span><ul class="toc-item"><li><span><a href="#Varying-number-of-splits-and-split-types" data-toc-modified-id="Varying-number-of-splits-and-split-types-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Varying number of splits and split types</a></span></li><li><span><a href="#Varying-number-of-workers-in-sync-mode" data-toc-modified-id="Varying-number-of-workers-in-sync-mode-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Varying number of workers in sync mode</a></span></li><li><span><a href="#Varying-number-of-workers-in-sync-and-async-mode" data-toc-modified-id="Varying-number-of-workers-in-sync-and-async-mode-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Varying number of workers in sync and async mode</a></span></li><li><span><a href="#Varying-query-type-and-filter" data-toc-modified-id="Varying-query-type-and-filter-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>Varying query type and filter</a></span></li><li><span><a href="#Varying-max-records-chunk-size" data-toc-modified-id="Varying-max-records-chunk-size-5.5"><span class="toc-item-num">5.5&nbsp;&nbsp;</span>Varying max-records chunk size</a></span></li><li><span><a href="#Select-and-run-with-your-own-parameters" data-toc-modified-id="Select-and-run-with-your-own-parameters-5.6"><span class="toc-item-num">5.6&nbsp;&nbsp;</span>Select and run with your own parameters</a></span></li></ul></li><li><span><a href="#Takeaways-and-Conclusion" data-toc-modified-id="Takeaways-and-Conclusion-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Takeaways and Conclusion</a></span></li><li><span><a href="#Further-Exploration-and-Resources" data-toc-modified-id="Further-Exploration-and-Resources-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Further Exploration and Resources</a></span></li></ul></div>

# Splitting Large Data Sets for Parallel Processing of Queries
This tutorial describes a scheme for dividing a large data set into equal splits for parallel processing of queries.

This notebook requires the Aerospike Database running locally with Java kernel and Aerospike Java Client. To create a Docker container that satisfies the requirements and holds a copy of Aerospike notebooks, visit the [Aerospike Notebooks Repo](https://github.com/aerospike-examples/interactive-notebooks).

## Introduction
In this notebook, we will implement a scheme to divide a set or the entire namespace in Aerospike into equal splits that can be queried in parallel. We will create a test framework that allows the splits to be processed over a range of parameters such as the number of workers, query and filter options, and processing mode. We will then run various combinations of splits, workers, and processing mode on the test data, and ensure the results remain the same for specific query and filter choices.

Please refer to the adjunct blog post [Processing Large Data Sets with Fine Grained Streams](../blog/fine-grained-streams) for details.

The specific topics covered in this notebook include:
- A scheme for dividing data equally over an arbitrary number of splits.
- The test framework to process the splits in parallel using a range of parameters such as the number of workers, query and filter options, and sync and async modes.
- Putting it to test with different values for splits, workers, query, filter, and processing mode.

## Prerequisites
This tutorial assumes familiarity with the following topics:
- [Hello World](hello_world.ipynb)

## Setup

### Ensure database is running
This notebook requires that Aerospike database is running. 

In [18]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;
IJava.getKernelInstance().getMagics().registerMagics(Shell.class);
%sh asd

### Download and install additional components.
Install the Java client.

In [19]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-client</artifactId>
    <version>6.1.0</version>
  </dependency>
</dependencies>

### Initialize Client
Initialize the client that can be used for both sync and async processing modes.

#### Initialize event loops for async processing mode
We will use async processiong using NIO event loops, but the other event loop types may also be used.

In [20]:
import java.util.concurrent.atomic.AtomicInteger;
import com.aerospike.client.async.EventPolicy;
import com.aerospike.client.async.EventLoops;
import com.aerospike.client.async.EventLoop;
import com.aerospike.client.async.Throttles;
import com.aerospike.client.async.Monitor;
import com.aerospike.client.async.NioEventLoops;
import com.aerospike.client.listener.RecordSequenceListener;

// initialize event loops 
final int NumLoops = 2;
final int CommandsPerEventLoop = 50;
final int DelayQueueSize = 50;

EventPolicy eventPolicy = new EventPolicy();
eventPolicy.maxCommandsInProcess = CommandsPerEventLoop;
eventPolicy.maxCommandsInQueue = DelayQueueSize;
EventLoops eventLoops = new NioEventLoops(eventPolicy, NumLoops);

// initialize event loop throttles
Throttles throttles = new Throttles(NumLoops, CommandsPerEventLoop);

System.out.format("Throttles initialized for %s loops with %s concurrent operations per loop.\n", 
                    NumLoops, CommandsPerEventLoop);


Throttles initialized for 2 loops with 50 concurrent operations per loop.


java.io.PrintStream@1afb8b3

#### Initialize client with event loops

In [21]:
import com.aerospike.client.AerospikeClient;
import com.aerospike.client.Host;
import com.aerospike.client.policy.ClientPolicy;

// initialize the client 
final int MaxConnPerNode = 10000; // adjust accordingly for max workers
ClientPolicy clientPolicy = new ClientPolicy();
clientPolicy.maxConnsPerNode = MaxConnPerNode;
clientPolicy.eventLoops = eventLoops;
int concurrentMax = CommandsPerEventLoop * NumLoops;
if (clientPolicy.maxConnsPerNode < concurrentMax) {
    clientPolicy.maxConnsPerNode = concurrentMax; 
}
Host[] hosts = Host.parseHosts("localhost", 3000); 

AerospikeClient client = new AerospikeClient(clientPolicy, hosts);

System.out.println("Initialized the client and connected to the cluster.");;

Initialized the client and connected to the cluster.


### Includes and Constants

In [22]:
import java.util.concurrent.atomic.AtomicInteger;
import java.util.concurrent.atomic.AtomicLong;
import com.aerospike.client.AerospikeException;
import com.aerospike.client.Bin;
import com.aerospike.client.Key;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.query.Filter;
import com.aerospike.client.query.PartitionFilter;
import com.aerospike.client.query.RecordSet;
import com.aerospike.client.query.Statement;
import com.aerospike.client.Record;
import com.aerospike.client.exp.Exp;
import com.aerospike.client.policy.Policy;
import com.aerospike.client.policy.QueryPolicy;
import com.aerospike.client.query.IndexType;
import com.aerospike.client.task.IndexTask;
import com.aerospike.client.ResultCode;

final String Namespace = "test";
final String Set = "query-splits";
final String KeyPrefix = "id-";
final Integer NumRecords = 100000;         // CHANGE TO THE REQUIRED NUMBER OF TEST DATA RECORDS

final int MIN_NUM_SPLITS = 1;              // at-least the requested number of splits
final int EXACT_NUM_SPLITS = 0;            // exactly the requested number of splits
final int MAX_NUM_SPLITS = -1;             // at-most the requested number of splits
final int PROCESSING_MODE_SYNC = 0;        // sync processing mode
final int PROCESSING_MODE_ASYNC = 1;       // async processing mode
final int PRIMARY_INDEX_QUERY = 0;         // primary index query (scan)
final int SECONDARY_INDEX_QUERY = 1;       // secondary index query - uses secondaryIndexQueryPredicate
final int QUERY_FILTER_NONE = 0;           // no additional filter in query
final int QUERY_FILTER_INCLUDE = 1;        // include the specified filter - uses includeQueryFilterExp


### Populate Test Data.
The test data consists of NumRecords records, each with a user key "id-\<i\>", an integer bin "bin1" with value i, and another integer bin with value 10*i, where 1 \<= i \<= NumRecords. An integer secondary index is created on "bin1".

In [36]:
// convenience function to initialize test data
void initializeTestData() {
    WritePolicy wpolicy = new WritePolicy();
    wpolicy.sendKey = true;
    for (int i = 1; i <= NumRecords; i++) {
        Key key = new Key(Namespace, Set, KeyPrefix+i);
        Bin bin1 = new Bin(new String("bin1"), i);
        Bin bin2 = new Bin(new String("bin2"), 10*i);
        client.put(wpolicy, key, bin1, bin2);
    }
}
initializeTestData();
System.out.println("Test data populated.");;

Test data populated.


In [24]:
Policy policy = new Policy();
policy.socketTimeout = 0; // Do not timeout on index create.

final String IndexName = "idx_bin1_number_idx";

try {
    IndexTask task = client.createIndex(policy, Namespace, Set, IndexName, 
                                        "bin1", IndexType.NUMERIC);
    task.waitTillComplete();
}
catch (AerospikeException ae) {
    if (ae.getResultCode() != ResultCode.INDEX_ALREADY_EXISTS) {
        throw ae;
    }
}

System.out.format("Created index %s on ns=%s set=%s bin=%s.", 
                                    IndexName, Namespace, Set, "bin1");;

Created index idx_bin1_number_idx on ns=test set=query-splits bin=bin1.

# Overview
The main sections consist of:
- Dividing partitions into N splits
- Parallel query framework
- Experimenting with parameter variations


# Dividing Partitions into Requested Splits
The main function assignSplits() returns an array of assignments across the requested number of splits. Each assignment has:

Partial or full partition assignment
A full partition assignment has the range of partitions , specified as the start partition id and count
A partial partition assignment has the partition id, modulo (division) factor, first sub-partition id, second sub-partition id (can be null)
The assignment parameters directly correspond to PartitionFilter specification in the queryPartitions API. For example, in Java, the query over full parttiions can be translated as follows:

PartitionFilter pFilter = PartitionFilter.range(startPartitionId, partitionCount) Statement stmt = new Statement(); stmt.setNamespace(namespace); stmt.setSetName(setName); stmt.setBinNames(bin1, ...); stmt.setMaxRecords(maxRecords); RecordSet rs = client.queryPartitions(qPolicy, qStatement, pFilter);

A query over sub-partition assignments is translated as follows:

PartitionFilter pFilter = PartitionFilter.id(partitionId) QueryPolicy qPolicy = new QueryPolicy(client.queryPolicyDefault); qPolicy.filterExp = Exp.build( Exp.or( Exp.eq(Exp.moduloDigest(moduloFactor), Exp.val(subPartitionId1)), Exp.eq(Exp.moduloDigest(moduloFactor), Exp.val(subPartitionId2))));

RecordSet rs = client.queryPartitions(qPolicy, qStatement, pFilter);

Each chunk of maxRecords retrieved is processed and the query is re-issued until pFilter.isDone() returns true.

In [25]:
class SplitAssignment {
    boolean isFullPartitionAssignment; // false for sub-partition assignments
    Integer fullPartitionStartId;
    Integer fullPartitionCount;
    Integer partialPartitionId;        // partition id of the sub-partition(s)
    Integer subPartitonModuloFactor;
    Integer firstSubpartition;         // always valid for a sub-partition assignment
    Integer secondSubpartition;        // can be null if only one sub-partition assigned
    
    SplitAssignment (int startId, int count) {
        this.isFullPartitionAssignment = true;
        this.fullPartitionStartId = startId;
        this.fullPartitionCount = count;
    }
    
    SplitAssignment(int partialPartitionId, int moduloFactor, 
                    int firstSubpartion, Integer secondSubpartition) {
        this.isFullPartitionAssignment = false;
        this.partialPartitionId = partialPartitionId;
        this.subPartitonModuloFactor = moduloFactor;
        this.firstSubpartition = firstSubpartion;
        this.secondSubpartition = secondSubpartition;                                   
    }
}  

final int FactorsOf4096[] = {1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096};

// split assignment types:
//       MAX_NUM_SPLITS At most requested number of splits (can be fewer), but same sized.
//       MIN_NUM_SPLITS At least requested number of splits (can be more), but same sized
//       EXACT_NUM_SPLITS Exactly requested number of splits, but can be different sized.

SplitAssignment[] assignSplits(int reqSplits, int splitType) {
    int numSplits = 0;
    SplitAssignment[] assignments = null;
    
    if (splitType == MAX_NUM_SPLITS) { // at most reqSplits
       if (reqSplits < 2*4096) {
            // Case 1: N < 2*4096: Full partition assignments
            // Closest approximation of splits, numSplits: factor of 4096 that is <= reqSplits
            for (int i = FactorsOf4096.length - 1; i >= 0; i--) {
                if (FactorsOf4096[i] <= reqSplits) {
                    numSplits = FactorsOf4096[i];
                    break;
                }
            }
            // Number of partitions in each split, count: 4096/numSplits
            int count = 4096 / numSplits;
            // Partitions in split i: i*count to (i+1)*n (upper bound exclusive)
            assignments = new SplitAssignment[numSplits];
            for (int i = 0; i < numSplits; i++) {
                assignments[i] = new SplitAssignment(i*count, count);
            }
        } else {        
            // Case 2: N >= 2*4096: Sub-partition assignment
            //Closest approximation of splits, numSplits: multiple of 4096 that is <= reqSplits
            numSplits = 4096 * (int)Math.floor((double)reqSplits/4096);
            //Number of sub-partitions per partition, s: numSplits/4096
            int moduloFactor = numSplits/4096;
            //Sub-partition in split i: (floor(i/s), i%s, s)
            assignments = new SplitAssignment[numSplits];
            for (int i = 0; i < numSplits; i++) {
                assignments[i] = new SplitAssignment((int)Math.floor((double)i/moduloFactor), moduloFactor,
                                                       i % moduloFactor, null);
            }
        }
        
    } else if (splitType == MIN_NUM_SPLITS) { // at least reqSplits
       if (reqSplits <= 4096) {
            // Case 1: N <= 4096: Full partition assignments
            // Closest approximation of splits, numSplits: factor of 4096 that is >= reqSplits
            for (int f: FactorsOf4096) {
                if (f >= reqSplits) {
                    numSplits = f;
                    break;
                }
            }
            // Number of partitions in each split, count: 4096/numSplits
            int count = 4096 / numSplits;
            // Partitions in split i: i*count to (i+1)*n (upper bound exclusive)
            assignments = new SplitAssignment[numSplits];
            for (int i = 0; i < numSplits; i++) {
                assignments[i] = new SplitAssignment(i*count, count);
            }
        } else {        
            // Case 2: N > 4096: Sub-partition assignment
            //Closest approximation of splits, numSplits: multiple of 4096 that is >= reqSplits
            numSplits = 4096 * (int)Math.ceil((double)reqSplits/4096);
            //Number of sub-partitions per partition, s: numSplits/4096
            int moduloFactor = numSplits/4096;
            //Sub-partition in split i: (floor(i/s), i%s, s)
            assignments = new SplitAssignment[numSplits];
            for (int i = 0; i < numSplits; i++) {
                assignments[i] = new SplitAssignment((int)Math.floor((double)i/moduloFactor), moduloFactor,
                                                       i % moduloFactor, null);
            }
        }

    } else { // exactly reqSplits
        if (reqSplits <= 4096) {
            // Case 1: N <= 4096
            // Each split is assigned full partitions only (maxModulo=1), 
            //.   with some may get an additional partition.
            // Evenly assign partitions over N splits.
            // Number of full partitions per split, n: floor(4096/N)
            int minPartitionsPerSplit = (int)Math.floor((double)4096/reqSplits);
            // Remaining partitions r are allocated in full to splits 0 to r-1
            int remainingPartitions = 4096 - minPartitionsPerSplit*reqSplits;
            // Assign minPartitionsPerSplit+1 to the first remainingPartitions splits, and
            //   minPartitionsPerSplit to the rest.
            int nextStartPartition = 0;
            assignments = new SplitAssignment[reqSplits];
            for (int i=0; i < reqSplits; i++){
                if (i < remainingPartitions) {
                    assignments[i] = new SplitAssignment(nextStartPartition, minPartitionsPerSplit+1);
                    nextStartPartition += minPartitionsPerSplit+1;
                } else {
                    assignments[i] = new SplitAssignment(nextStartPartition, minPartitionsPerSplit);
                    nextStartPartition += minPartitionsPerSplit;
                }
            }
        } else {
            // Case 2: N > 4096
            // Each split is assigned one or two subpartitions in the same partition.
            // Divide each partition into m (modulo-factor) subparittions: ceiling(N/4096) – 
            //    the ceiling function ensures assignment across all N splits.
            int moduloFactor = (int)Math.ceil((double)reqSplits/4096);
            // 4096*moduloFactor sub-partitions are available to be assigned across reqSplits
            // reqSplits sub-partitions are assigned one per split
            // Remaining sub-partitions r = (4096*moduloFactor - reqSplits) are allocated to splits 0 to r-1.
            int remainingSubPartitions = 4096*moduloFactor - reqSplits;
            // Assign 2 sub-partitions to the first remainingSubPartitions splits while ensuring both
            //.  sub-partitions are from the same partition, and 1 sub-partition to the rest.
            int twoSubsSplitIndex = 0; // varies from 0 to remainingSubPartitions-1
            int oneSubSplitIndex = remainingSubPartitions; // varies from remainingSubPartitions to reqSplits
            assignments = new SplitAssignment[reqSplits];
            for (int i=0; i < 4096; i++) {
                int subPartition = 0;
                while (subPartition < moduloFactor) {
                    if (twoSubsSplitIndex < remainingSubPartitions &&
                            subPartition+1 < moduloFactor) { // both subpartitions must be in the same partition
                        assignments[twoSubsSplitIndex] = new SplitAssignment(i, moduloFactor,
                                                           subPartition, subPartition+1);
                        twoSubsSplitIndex++;
                        subPartition += 2; 
                    } else {
                        // single partition assignment
                        assignments[oneSubSplitIndex] = new SplitAssignment(i, moduloFactor,
                                                           subPartition, null);
                        oneSubSplitIndex++;
                        subPartition++; 
                    }
                }
            }
        }
    }
    
    return assignments;
}

void assignmentSummary(SplitAssignment[] sa) {
    System.out.format("%d splits: \n", sa.length);
    if (sa[0].isFullPartitionAssignment) {
        int i = 0;
        System.out.format("\tSplit: %s, start: %s, count: %s\n", 
                    i, sa[i].fullPartitionStartId, sa[i].fullPartitionCount);
        int initialCount = sa[0].fullPartitionCount;
        while (i < sa.length && sa[i].fullPartitionCount == initialCount) i++;
        System.out.format("\tSplit: %s, start: %s, count: %s\n", 
                    i-1, sa[i-1].fullPartitionStartId, sa[i-1].fullPartitionCount); 
        if (i < sa.length) {
            System.out.format("\tSplit: %s, start: %s, count: %s\n", 
                        i, sa[i].fullPartitionStartId, sa[i].fullPartitionCount);
            i = sa.length-1;
            System.out.format("\tSplit: %s, start: %s, count: %s\n", 
                        i, sa[i].fullPartitionStartId, sa[i].fullPartitionCount);            
        }
    } else {
        int i = 0;
        System.out.format("\tSplit: %s, partial: %s, first: %s, second: %s, modulo: %s\n", 
            i, sa[i].partialPartitionId, sa[i].firstSubpartition, sa[i].secondSubpartition, sa[i].subPartitonModuloFactor); 
        Integer initialSecondPartition = sa[0].secondSubpartition;
        while (i < sa.length && (initialSecondPartition == null || sa[i].secondSubpartition != null)) i++;
        System.out.format("\tSplit: %s, partial: %s, first: %s, second: %s, modulo: %s\n", 
            i-1, sa[i-1].partialPartitionId, sa[i-1].firstSubpartition, sa[i-1].secondSubpartition, sa[i-1].subPartitonModuloFactor); 
        if (i < sa.length) {
            System.out.format("\tSplit: %s, partial: %s, first: %s, second: %s, modulo: %s\n", 
                i, sa[i].partialPartitionId, sa[i].firstSubpartition, sa[i].secondSubpartition, sa[i].subPartitonModuloFactor); 
            i = sa.length-1;
            System.out.format("\tSplit: %s, partial: %s, first: %s, second: %s, modulo: %s\n", 
                i, sa[i].partialPartitionId, sa[i].firstSubpartition, sa[i].secondSubpartition, sa[i].subPartitonModuloFactor); 
        }
    }    
}

# Parallel Query Framework
The parallel processing test framework works as follows:
- Create N splits. This can be varied over a wide range such as 1-50K, along with the type parameter over AtLeast, AtMost, and Exact values.
- Spawn W worker threads. This can be varied over a wide range such as 1-10K or more if the container allows it. All threads start processing splits at the same time, and the main thread waits until all threads finish.
- A worker executes a loop:
    - While threr is an unprocessed split available:
    - Get a split from the available splits.
    - Process the split:
        - While there are more records to be processed in the split, in sync and async mode:
        - Execute a query to retrieve max-records. The query can be a primary-index query or a secondary-index query, and with or without a fiter expression.
        - Process the chunk (count the records, and compute the aggregate sum).
- The total record count and aggregate sum is printed out at the end, which should be the same for a given query and filter expression

The code below implements the parallel processing framework that allows interesting parameters to be selected for each run.

## Global parameters
The following parameters are implemented as global variables for convenience. They can be set to appropriate desired values before a run.

In [26]:
// Global parameters - set to specific values later before a run
int ProcessingMode;     // set to PROCESSING_MODE_SYNC or PROCESSING_MODE_ASYNC
int QueryType;          // PRIMARY_INDEX_QUERY or SECONDARY_INDEX_QUERY
int QueryFilter;        // QUERY_FILTER_NONE or QUERY_FILTER_INCLUDE
int ChunkSize;          // max number of records retrieved in a chunk
Filter SecondaryIndexQueryPredicate; // sindex query predicate, used when QueryType = SECONDARY_INDEX_QUERY
Exp IncludeQueryFilterExp; // filter expression, used when QueryFilter = QUERY_FILTER_INCLUDE


In [41]:
// Implementation of the parallel query framework

AtomicInteger recCount = new AtomicInteger();    // total number of records processed
AtomicLong recSum = new AtomicLong();            // sum across all records
AtomicInteger workerId = new AtomicInteger();    // worker id
AtomicInteger nextSplit = new AtomicInteger();   // next unclaimed split
CountDownLatch startLineLatch;                   // forces all workers to start at the same time

Statement stmt = new Statement();
stmt.setNamespace(Namespace);
stmt.setSetName(Set);
stmt.setBinNames("bin1", "bin2");

class Worker extends Thread {
    static SplitAssignment[] assignments; // splits to be processed
    static int[] workerRecCounts;         // records processed by each worker
        
    @Override public void run(){
        int myId = workerId.getAndIncrement();    
        try {
            startLineLatch.countDown(); // count down
            startLineLatch.await();     // all workers start when count reaches zero
        }
        catch (Exception e) {
           System.out.format("%s", e);
        }
        //System.out.println(Thread.currentThread().getName()
        //            + ": " + myId);
        int numSplits = Worker.assignments.length;
        // loop to get the next available split until all splits are gone
        for (int split = nextSplit.getAndIncrement(); split < numSplits; split = nextSplit.getAndIncrement()) {
            SplitAssignment sa = Worker.assignments[split]; // split to process
            PartitionFilter pFilter;
            QueryPolicy qPolicy = new QueryPolicy(client.queryPolicyDefault);
            qPolicy.filterExp = null;
            // define the query partition filter
            if (sa.isFullPartitionAssignment) { // if full partitions, assign the range
                pFilter = PartitionFilter.range(sa.fullPartitionStartId, sa.fullPartitionCount);
                if (QueryFilter == QUERY_FILTER_INCLUDE) { 
                    qPolicy.filterExp = Exp.build(IncludeQueryFilterExp); //  filter expression is specified in policy
                }
            } else { // if partial partitions, define the digest-modulo filter expression
                int moduloFactor = sa.subPartitonModuloFactor;
                pFilter = PartitionFilter.id(sa.partialPartitionId); 
                Exp firstSubpartitionExp = Exp.eq(Exp.digestModulo(moduloFactor), Exp.val(sa.firstSubpartition));
                Exp subpartitionExp = firstSubpartitionExp;
                if (sa.secondSubpartition != null) { // a split has a second sub-partition
                    subpartitionExp = Exp.or(
                                        firstSubpartitionExp, 
                                        Exp.eq(Exp.digestModulo(moduloFactor), Exp.val(sa.secondSubpartition)));
                }
                Exp queryFilterExp = subpartitionExp;
                if (QueryFilter == QUERY_FILTER_INCLUDE) { // AND with the query's filter expression
                    queryFilterExp = Exp.and(
                                        subpartitionExp,
                                        IncludeQueryFilterExp);
                }
                qPolicy.filterExp = Exp.build(queryFilterExp); //  filter expression is specified in policy
            }
            stmt.setMaxRecords(ChunkSize);  // chunked retrieval of query results 
            stmt.setFilter(null);            // this query filter is null for a primary-index query
            if (QueryType == SECONDARY_INDEX_QUERY) {
                stmt.setFilter(SecondaryIndexQueryPredicate); // set secondary-index query predicate here
            }
            if (ProcessingMode == PROCESSING_MODE_SYNC) { // select the processing mode
                Worker.workerRecCounts[myId] += this.processSync(qPolicy, stmt, pFilter); 
            } else {
                Worker.workerRecCounts[myId] += this.processAsync(qPolicy, stmt, pFilter);
            }
            try {
                Thread.sleep(1); // allow other threads to run
            }
            catch (Exception e) {
               System.out.format("%s", e);
            }
        }
    }
    
    int processSync(QueryPolicy qPolicy, Statement stmt, PartitionFilter pFilter) {
        int splitRecs = 0; // tracks number of records in this split
        long splitSum = 0; // tracks sum of records in this split
        // retrieve chunks of records in a loop
        while (! pFilter.isDone()) {
            RecordSet rs = client.queryPartitions(qPolicy, stmt, pFilter);
            try {
                while (rs.next()) {
                    splitRecs++;
                    Record rec = rs.getRecord();
                    splitSum += (long)rec.bins.get("bin1");             
                }
                //System.out.println("Records returned: " + count);
            }
            finally {
                rs.close();
            }
        }
        recCount.addAndGet(splitRecs);
        recSum.addAndGet(splitSum);
        return splitRecs;
    }
        
    int processAsync(QueryPolicy qPolicy, Statement stmt, PartitionFilter pFilter) {
        AtomicInteger splitRecs = new AtomicInteger(0);
        // Query all pages of records.
        while (! pFilter.isDone()) {
            // query monitor to await/notify completion of each chunk
            Monitor queryMonitor = new Monitor();
            // submit async operation with throttle by waiting for an available slot
            EventLoop eventLoop = eventLoops.next();
            int eventLoopIndex = eventLoop.getIndex();
            if (throttles.waitForSlot(eventLoopIndex, 1)) { 
                try {
                    client.queryPartitions(eventLoop, new RecordSequenceListener() {
                            public void onRecord(Key key, Record record) throws AerospikeException {
                                splitRecs.incrementAndGet();
                                recSum.addAndGet((long)record.bins.get("bin1"));
                             }
                             
                            public void onSuccess() {
                                throttles.addSlot(eventLoopIndex, 1);
                                queryMonitor.notifyComplete();
                            }

                            public void onFailure(AerospikeException e) {
                                throttles.addSlot(eventLoopIndex, 1);
                                System.out.format("Error: query failed with exception - %s", e);
                                queryMonitor.notifyComplete();
                            }                    
                        }, 
                        qPolicy, stmt, pFilter);
                }
                catch (Exception e) {
                   System.out.format("Error: exception in record sequence listener - %s\n", e.getMessage());
                }
            }
            queryMonitor.waitTillComplete();
        }
        int count = splitRecs.get();
        recCount.addAndGet(count);
        return count;
    }
}

void workerSummary() {
    System.out.print("Records by worker (~50 worker sample): ");
    float step = (Worker.workerRecCounts.length-1)/50;
    for (int i = 0; i < Worker.workerRecCounts.length; i += 1+step) {
        System.out.format("%d:%d ", i, Worker.workerRecCounts[i]);
    }
    System.out.println("");
}

void processSplits(SplitAssignment[] splitAssignments, int numWorkers) {
    System.out.format("%d worker threads.\n", numWorkers);
    Worker.assignments = splitAssignments;
    Worker.workerRecCounts = new int[numWorkers];
    workerId.set(0);
    nextSplit.set(0);
    recCount.set(0);
    recSum.set(0); 
    startLineLatch = new CountDownLatch(numWorkers);
    List<Worker> workers = new ArrayList<>();
    for (int i = 0; i < numWorkers; i++) {
        Worker worker = new Worker();
        worker.setName("Worker " + i);
        workers.add(worker);
        worker.start();
    }
    long startTime = System.currentTimeMillis();
    try {
        for (Worker worker: workers) {
            worker.join();
        }
    }
    catch (Exception e) {
       System.out.format("%s", e);
    }
    long endTime = System.currentTimeMillis();
    System.out.format("Record count is %d, sum is %d.\n", recCount.get(), recSum.get());
    System.out.println("That took " + (endTime - startTime)/1000 + " seconds.");
}

# Running Parallel Queries with Interesting Parameter Variations
The key things to note are:
- Split assignments - a summary can be printed out optionally as seen below.
- Number of records and aggregated sum should be the same for the same query/filter setting, confirming all records are processed correctly.
- Performance indicated by the time it took to execute a query
- Distribution of records processed by worker - a summary can be printed out optionally as seen below.

In [39]:
// set the base global parameter values - can be changed here or before a run in the cells below
ProcessingMode = PROCESSING_MODE_SYNC;
QueryType = PRIMARY_INDEX_QUERY;
QueryFilter = QUERY_FILTER_NONE;
ChunkSize = 20;

// the following filter expression chooses records with even values of bin1 - CAN CHANGE TO A DIFFERENT EXPRESSION
//    used when QueryFilter = QUERY_FILTER_INCLUDE
IncludeQueryFilterExp = Exp.eq(Exp.mod(Exp.intBin("bin1"), Exp.val(2)), Exp.val(0));

// the following query predicate is a range filter for records with 50001 <= bin1 <= 100000 - CAN CHANGE TO A DIFFERENT QUERY PREDICATE
//    used when QueryType = SECONDARY_INDEX_QUERY
SecondaryIndexQueryPredicate = Filter.range("bin1", 50001, 100000);



## Varying number of splits and split types
Vary the number of splits from 1-50000 acoss all split types.

In [29]:
int NumWorkers = 100;

System.out.format("Requested splits: %d, type: %s\n", 2, "MIN_NUM_SPLITS");
SplitAssignment[] splitAssignments = assignSplits(2, MIN_NUM_SPLITS);
assignmentSummary(splitAssignments);
processSplits(splitAssignments, NumWorkers);
System.out.println();

System.out.format("Requested splits: %d, type: %s\n", 10, "MIN_NUM_SPLITS");
splitAssignments = assignSplits(10, MIN_NUM_SPLITS);
assignmentSummary(splitAssignments);
processSplits(splitAssignments, NumWorkers);
System.out.println();

System.out.format("Requested splits: %d, type: %s\n", 100, "EXACT_NUM_SPLITS");
splitAssignments = assignSplits(100, EXACT_NUM_SPLITS);
assignmentSummary(splitAssignments);
processSplits(splitAssignments, NumWorkers);
workerSummary();
System.out.println();

System.out.format("Requested splits: %d, type: %s\n", 1000, "MAX_NUM_SPLITS");
splitAssignments = assignSplits(1000, MAX_NUM_SPLITS);
assignmentSummary(splitAssignments);
processSplits(splitAssignments, NumWorkers);
workerSummary();
System.out.println();

System.out.format("Requested splits: %d, type: %s\n", 5000, "MAX_NUM_SPLITS");
splitAssignments = assignSplits(5000, MAX_NUM_SPLITS);
assignmentSummary(splitAssignments);
processSplits(splitAssignments, NumWorkers);
workerSummary();
System.out.println();

System.out.format("Requested splits: %d, type: %s\n", 10000, "EXACT_NUM_SPLITS");
splitAssignments = assignSplits(10000, EXACT_NUM_SPLITS);
assignmentSummary(splitAssignments);
processSplits(splitAssignments, NumWorkers);
workerSummary();
System.out.println();

System.out.format("Requested splits: %d, type: %s\n", 50000, "MAX_NUM_SPLITS");
splitAssignments = assignSplits(50000, MAX_NUM_SPLITS);
assignmentSummary(splitAssignments);
processSplits(splitAssignments, NumWorkers);
workerSummary();
System.out.println();


Requested splits: 2, type: MIN_NUM_SPLITS
2 splits: 
	Split: 0, start: 0, count: 2048
	Split: 1, start: 2048, count: 2048
100 worker threads.
Record count is 100000, sum is 5000050000.
That took 40 seconds.

Requested splits: 10, type: MIN_NUM_SPLITS
16 splits: 
	Split: 0, start: 0, count: 256
	Split: 15, start: 3840, count: 256
100 worker threads.
Record count is 100000, sum is 5000050000.
That took 6 seconds.

Requested splits: 100, type: EXACT_NUM_SPLITS
100 splits: 
	Split: 0, start: 0, count: 41
	Split: 95, start: 3895, count: 41
	Split: 96, start: 3936, count: 40
	Split: 99, start: 4056, count: 40
100 worker threads.
Record count is 100000, sum is 5000050000.
That took 3 seconds.
Records by worker (~50 worker sample): 0:991 2:1046 4:1014 6:1047 8:999 10:1000 12:1000 14:997 16:948 18:1007 20:1017 22:990 24:1015 26:999 28:1005 30:1050 32:953 34:985 36:964 38:970 40:959 42:1041 44:993 46:1015 48:906 50:963 52:975 54:993 56:958 58:1023 60:1021 62:956 64:1014 66:947 68:944 70:990 72:1

## Varying number of workers in sync mode
Vary the number of workers from 10 to 5000.

In [42]:
int ReqSplits = 10000;

SplitAssignment[] splitAssignments = assignSplits(ReqSplits, EXACT_NUM_SPLITS);
assignmentSummary(splitAssignments);
processSplits(splitAssignments, 10);
workerSummary();
System.out.println();

splitAssignments = assignSplits(ReqSplits, EXACT_NUM_SPLITS);
//assignmentSummary(splitAssignments);
processSplits(splitAssignments, 100);
workerSummary();
System.out.println();

splitAssignments = assignSplits(ReqSplits, EXACT_NUM_SPLITS);
//assignmentSummary(splitAssignments);
processSplits(splitAssignments, 1000);
workerSummary();
System.out.println();

splitAssignments = assignSplits(ReqSplits, EXACT_NUM_SPLITS);
//assignmentSummary(splitAssignments);
processSplits(splitAssignments, 5000);
workerSummary();
System.out.println();


10000 splits: 
	Split: 0, partial: 0, first: 0, second: 1, modulo: 3
	Split: 2287, partial: 2287, first: 0, second: 1, modulo: 3
	Split: 2288, partial: 0, first: 2, second: null, modulo: 3
	Split: 9999, partial: 4095, first: 2, second: null, modulo: 3
10 worker threads.
Record count is 100000, sum is 5000050000.
That took 4 seconds.
Records by worker (~50 worker sample): 0:10155 1:10143 2:9999 3:10039 4:9923 5:10316 6:9852 7:9979 8:9899 9:9695 

100 worker threads.
Record count is 100000, sum is 5000050000.
That took 4 seconds.
Records by worker (~50 worker sample): 0:973 2:1054 4:934 6:1102 8:1012 10:901 12:1045 14:966 16:1121 18:1063 20:1063 22:924 24:950 26:894 28:989 30:971 32:987 34:966 36:1067 38:1088 40:1092 42:981 44:1032 46:1051 48:1036 50:956 52:1056 54:989 56:900 58:882 60:928 62:987 64:991 66:1134 68:1063 70:941 72:1092 74:1020 76:1083 78:1110 80:895 82:879 84:940 86:1012 88:1085 90:1128 92:1049 94:909 96:962 98:960 

1000 worker threads.
Record count is 100000, sum is 5000

## Varying number of workers in sync and async mode
Vary the number of workers from 10 to 5000 in sync and async mode.

In [31]:
int ReqSplits = 10000;

ProcessingMode = PROCESSING_MODE_SYNC;
System.out.format("Processing mode: SYNC\n");
SplitAssignment[] splitAssignments = assignSplits(ReqSplits, EXACT_NUM_SPLITS);
assignmentSummary(splitAssignments);
processSplits(splitAssignments, 10);
workerSummary();
System.out.println();

ProcessingMode = PROCESSING_MODE_ASYNC;
System.out.format("Processing mode: ASYNC\n");
splitAssignments = assignSplits(ReqSplits, EXACT_NUM_SPLITS);
//assignmentSummary(splitAssignments);
processSplits(splitAssignments, 10);
workerSummary();
System.out.println();

ProcessingMode = PROCESSING_MODE_SYNC;
System.out.format("Processing mode: SYNC\n");
SplitAssignment[] splitAssignments = assignSplits(ReqSplits, EXACT_NUM_SPLITS);
//assignmentSummary(splitAssignments);
processSplits(splitAssignments, 100);
workerSummary();
System.out.println();

ProcessingMode = PROCESSING_MODE_ASYNC;
System.out.format("Processing mode: ASYNC\n");
splitAssignments = assignSplits(ReqSplits, EXACT_NUM_SPLITS);
//assignmentSummary(splitAssignments);
processSplits(splitAssignments, 100);
workerSummary();
System.out.println();

ProcessingMode = PROCESSING_MODE_SYNC;
System.out.format("Processing mode: SYNC\n");
SplitAssignment[] splitAssignments = assignSplits(ReqSplits, EXACT_NUM_SPLITS);
//assignmentSummary(splitAssignments);
processSplits(splitAssignments, 1000);
workerSummary();
System.out.println();

ProcessingMode = PROCESSING_MODE_ASYNC;
System.out.format("Processing mode: ASYNC\n");
splitAssignments = assignSplits(ReqSplits, EXACT_NUM_SPLITS);
//assignmentSummary(splitAssignments);
processSplits(splitAssignments, 1000);
workerSummary();
System.out.println();

ProcessingMode = PROCESSING_MODE_SYNC;
System.out.format("Processing mode: SYNC\n");
SplitAssignment[] splitAssignments = assignSplits(ReqSplits, EXACT_NUM_SPLITS);
//assignmentSummary(splitAssignments);
processSplits(splitAssignments, 5000);
workerSummary();
System.out.println();

ProcessingMode = PROCESSING_MODE_ASYNC;
System.out.format("Processing mode: ASYNC\n");
splitAssignments = assignSplits(ReqSplits, EXACT_NUM_SPLITS);
//assignmentSummary(splitAssignments);
processSplits(splitAssignments, 5000);
workerSummary();
System.out.println();


Processing mode: SYNC
10000 splits: 
	Split: 0, partial: 0, first: 0, second: 1, modulo: 3
	Split: 2287, partial: 2287, first: 0, second: 1, modulo: 3
	Split: 2288, partial: 0, first: 2, second: null, modulo: 3
	Split: 9999, partial: 4095, first: 2, second: null, modulo: 3
10 worker threads.
Record count is 100000, sum is 5000050000.
That took 6 seconds.
Records by worker (~50 worker sample): 0:9974 1:10131 2:10240 3:10020 4:10041 5:9992 6:10007 7:10025 8:9861 9:9709 

Processing mode: ASYNC
10 worker threads.
Record count is 100000, sum is 5000050000.
That took 3 seconds.
Records by worker (~50 worker sample): 0:10025 1:9966 2:9847 3:10223 4:10167 5:9934 6:9956 7:9914 8:9845 9:10123 

Processing mode: SYNC
100 worker threads.
Record count is 100000, sum is 5000050000.
That took 5 seconds.
Records by worker (~50 worker sample): 0:1019 2:990 4:1064 6:982 8:986 10:999 12:1025 14:1021 16:965 18:1004 20:1014 22:1015 24:968 26:997 28:997 30:1062 32:1006 34:973 36:1017 38:970 40:977 42:1008 

## Varying query type and filter
Primary-index and sexondary-index queries with and without a filter expression. Note the default secondary-index query predicate will process half the records and yield half the sum value. The default filter expression, in turn, will also process half the records and yield half the sum of the filterless query.

In [32]:
// the following filter expression chooses records with even values of bin1 - CAN CHANGE TO A DIFFERENT EXPRESSION
//    used when QueryFilter = QUERY_FILTER_INCLUDE
IncludeQueryFilterExp = Exp.eq(Exp.mod(Exp.intBin("bin1"), Exp.val(2)), Exp.val(0));

// the following query predicate is a range filter for records with 50001 <= bin1 <= 100000 - CAN CHANGE TO A DIFFERENT QUERY PREDICATE
//    used when QueryType = SECONDARY_INDEX_QUERY
SecondaryIndexQueryPredicate = Filter.range("bin1", 25001, 75000);

int ReqSplits = 1000;
int NumWorkers = 100;
ProcessingMode = PROCESSING_MODE_SYNC;

QueryType = PRIMARY_INDEX_QUERY;
QueryFilter = QUERY_FILTER_NONE;
System.out.format("Primary-index query with no filter\n");
splitAssignments = assignSplits(ReqSplits, EXACT_NUM_SPLITS);
assignmentSummary(splitAssignments);
processSplits(splitAssignments, NumWorkers);
workerSummary();
System.out.println();

QueryType = PRIMARY_INDEX_QUERY;
QueryFilter = QUERY_FILTER_INCLUDE;
System.out.format("Primary-index query with filter expression\n");
splitAssignments = assignSplits(ReqSplits, EXACT_NUM_SPLITS);
//assignmentSummary(splitAssignments);
processSplits(splitAssignments, NumWorkers);
//workerSummary();
System.out.println();

QueryType = SECONDARY_INDEX_QUERY;
QueryFilter = QUERY_FILTER_NONE;
System.out.format("Secondary-index query with no filter\n");
splitAssignments = assignSplits(ReqSplits, EXACT_NUM_SPLITS);
//assignmentSummary(splitAssignments);
processSplits(splitAssignments, NumWorkers);
//workerSummary();
System.out.println();

QueryType = SECONDARY_INDEX_QUERY;
QueryFilter = QUERY_FILTER_INCLUDE;
System.out.format("Secondary-index query with filter expression\n");
splitAssignments = assignSplits(ReqSplits, EXACT_NUM_SPLITS);
//assignmentSummary(splitAssignments);
processSplits(splitAssignments, NumWorkers);
//workerSummary();
System.out.println();



Primary-index query with no filter
1000 splits: 
	Split: 0, start: 0, count: 5
	Split: 95, start: 475, count: 5
	Split: 96, start: 480, count: 4
	Split: 999, start: 4092, count: 4
100 worker threads.
Record count is 100000, sum is 5000050000.
That took 2 seconds.
Records by worker (~50 worker sample): 0:1035 2:1148 4:1074 6:1186 8:1024 10:1020 12:919 14:827 16:1105 18:947 20:1318 22:888 24:974 26:867 28:801 30:841 32:899 34:1045 36:1025 38:914 40:1176 42:808 44:985 46:902 48:1485 50:911 52:980 54:952 56:1009 58:716 60:861 62:1169 64:1226 66:810 68:1105 70:725 72:901 74:841 76:1064 78:1062 80:1070 82:970 84:950 86:1037 88:878 90:1104 92:961 94:955 96:821 98:1029 

Primary-index query with filter expression
100 worker threads.
Record count is 50000, sum is 2500050000.
That took 1 seconds.

Secondary-index query with no filter
100 worker threads.
Record count is 50540, sum is 2526213401.
That took 1 seconds.

Secondary-index query with filter expression
100 worker threads.
Record count is

## Varying max-records chunk size
Vary the chunk size from 10 to 500.

In [33]:
int ReqSplits = 100;
int NumWorkers = 50;
ProcessingMode = PROCESSING_MODE_ASYNC;
QueryType = PRIMARY_INDEX_QUERY;
QueryFilter = QUERY_FILTER_NONE;

ChunkSize = 10;
System.out.format("Max-records chunk size: %d\n", ChunkSize);
splitAssignments = assignSplits(ReqSplits, EXACT_NUM_SPLITS);
assignmentSummary(splitAssignments);
processSplits(splitAssignments, NumWorkers);
workerSummary();
System.out.println();

ChunkSize = 25;
System.out.format("Max-records chunk size: %d\n", ChunkSize);
splitAssignments = assignSplits(ReqSplits, EXACT_NUM_SPLITS);
//assignmentSummary(splitAssignments);
processSplits(splitAssignments, NumWorkers);
//workerSummary();
System.out.println();

ChunkSize = 100;
System.out.format("Max-records chunk size: %d\n", ChunkSize);
splitAssignments = assignSplits(ReqSplits, EXACT_NUM_SPLITS);
//assignmentSummary(splitAssignments);
processSplits(splitAssignments, NumWorkers);
//workerSummary();
System.out.println();

ChunkSize = 300;
System.out.format("Max-records chunk size: %d\n", ChunkSize);
splitAssignments = assignSplits(ReqSplits, EXACT_NUM_SPLITS);
//assignmentSummary(splitAssignments);
processSplits(splitAssignments, NumWorkers);
//workerSummary();
System.out.println();

ChunkSize = 500;
System.out.format("Max-records chunk size: %d\n", ChunkSize);
splitAssignments = assignSplits(ReqSplits, EXACT_NUM_SPLITS);
//assignmentSummary(splitAssignments);
processSplits(splitAssignments, NumWorkers);
//workerSummary();
System.out.println();


Max-records chunk size: 10
100 splits: 
	Split: 0, start: 0, count: 41
	Split: 95, start: 3895, count: 41
	Split: 96, start: 3936, count: 40
	Split: 99, start: 4056, count: 40
50 worker threads.
Record count is 100000, sum is 5000050000.
That took 4 seconds.
Records by worker (~50 worker sample): 0:1933 1:2080 2:2025 3:2061 4:2081 5:2088 6:2061 7:1971 8:2040 9:2007 10:2021 11:2006 12:1982 13:2012 14:1945 15:2002 16:2018 17:1951 18:1999 19:2008 20:1981 21:1984 22:1941 23:1985 24:2021 25:1954 26:2005 27:1987 28:2043 29:1999 30:2012 31:2106 32:1930 33:2091 34:1975 35:1950 36:1940 37:2046 38:1917 39:1892 40:1982 41:1927 42:2061 43:2062 44:1968 45:1986 46:2021 47:1943 48:1943 49:2057 

Max-records chunk size: 25
50 worker threads.
Record count is 100000, sum is 5000050000.
That took 2 seconds.

Max-records chunk size: 100
50 worker threads.
Record count is 100000, sum is 5000050000.
That took 0 seconds.

Max-records chunk size: 300
50 worker threads.
Record count is 100000, sum is 500005000

## Select and run with your own parameters
Experiment with the values in the following cell and see what results you get.

In [34]:
ProcessingMode = PROCESSING_MODE_SYNC;
QueryType = PRIMARY_INDEX_QUERY;
QueryFilter = QUERY_FILTER_NONE;
ChunkSize = 20;

// the following filter expression chooses records with even values of bin1 - CAN CHANGE TO A DIFFERENT EXPRESSION
//    used when QueryFilter = QUERY_FILTER_INCLUDE
IncludeQueryFilterExp = Exp.eq(Exp.mod(Exp.intBin("bin1"), Exp.val(2)), Exp.val(0));

// the following query predicate is a range filter for records with 50001 <= bin1 <= 100000 - CAN CHANGE TO A DIFFERENT QUERY PREDICATE
//    used when QueryType = SECONDARY_INDEX_QUERY
SecondaryIndexQueryPredicate = Filter.range("bin1", 50001, 100000);

int ReqSplits = 1000;
int SplitType = EXACT_NUM_SPLITS;
int NumWorkers = 100;

System.out.format("Running with parameter values (specify)\n");
SplitAssignment[] splitAssignments = assignSplits(ReqSplits, EXACT_NUM_SPLITS);
assignmentSummary(splitAssignments);
processSplits(splitAssignments, NumWorkers);
workerSummary();



Running with parameter values (specify)
1000 splits: 
	Split: 0, start: 0, count: 5
	Split: 95, start: 475, count: 5
	Split: 96, start: 480, count: 4
	Split: 999, start: 4092, count: 4
100 worker threads.
Record count is 100000, sum is 5000050000.
That took 2 seconds.
Records by worker (~50 worker sample): 0:751 2:1000 4:902 6:919 8:841 10:888 12:1347 14:954 16:1213 18:782 20:914 22:1008 24:886 26:945 28:926 30:916 32:781 34:794 36:864 38:866 40:887 42:795 44:911 46:972 48:1092 50:1234 52:1151 54:908 56:853 58:769 60:884 62:1319 64:1070 66:984 68:1115 70:1391 72:867 74:793 76:1045 78:1101 80:971 82:1141 84:866 86:1082 88:873 90:1245 92:920 94:1128 96:1074 98:902 


# Takeaways and Conclusion
The ability to retrieve a large data set in parallel streams for processing in a large number of workers is essential to high throughput applications. Aerospike provides mechanisms to divide a large data set over an arbitrary number of splits and process each split in smaller chunks at a time. The notebook implements an algorithm for split assignments, and provides a framework for parallel processing a query over splits. It also demonstrates parallel processing using varying values of splits, workers, query and filter, chunk size, and processing mode.

# Further Exploration and Resources
Here are some links for further exploration

Resources
- [Processing Large Data Sets with Fine Grained Streams](https://deveveloper.aerospike.com/blog/fine-grained-streams) (blog post)
- [Aerospike Developer Hub](https://deveveloper.aerospike.com)
